In [1]:
const namespaces = require('./namespaces')
const sparqljs = require('sparqljs')
const parser = new sparqljs.Parser(namespaces)

const eachFileInDir = require('./eachFileInDir')

const EN_SPARQL = 'data/sparql_queries_enwiki'
const DE_SPARQL = 'data/sparql_queries_dewiki'

## SELECT (silly)

In [2]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    if (query.toLowerCase().includes('select')) occurences++

    total++
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

997 997


## OPTIONAL

In [3]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const hasOptional = parsedQuery.where.some((clause) => {
        return clause.type === 'optional'
    })
    if (hasOptional) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 144


## FILTER

In [4]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const hasFilter = parsedQuery.where.some((clause) => {
        return clause.type === 'filter'
    })
    if (hasFilter) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 373


## ORDER BY

In [5]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    if (parsedQuery.order) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 98


## DISTINCT

In [6]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    if (parsedQuery.distinct) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 46


## VALUES

In [7]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const hasValues = parsedQuery.where.some((clause) => {
        return clause.type === 'values'
    })
    if (hasValues) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 56


## UNION

In [8]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const hasUnion = parsedQuery.where.some((clause) => {
        return clause.type === 'union'
    })
    if (hasUnion) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 86


## MINUS

In [9]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const hasMinus = parsedQuery.where.some((clause) => {
        return clause.type === 'minus'
    })
    if (hasMinus) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 32


## Subqueries

In [10]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const hasSubquery = parsedQuery.where.some((clause) => {
        return clause.type === 'query'
    })
    if (hasSubquery) occurences++
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 16


## Multiple Subject Queries

In [11]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const triples = parsedQuery.where.reduce(
        (triples, clause) => clause.type === 'bgp' ? triples.concat(clause.triples) : triples,
        []
    )
    const subjects = triples.map(triple => triple.subject)
    const objects = triples.map(triple => triple.object)
    const hasMultipleSubjects = subjects.some((subject) => {
        return typeof subject === 'string' && subject.startsWith('?') && objects.includes(subject)
    })
    
    if (hasMultipleSubjects) occurences++
    
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 297


## Property Path

In [12]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const triples = parsedQuery.where.reduce(
        (triples, clause) => clause.type === 'bgp' ? triples.concat(clause.triples) : triples,
        []
    )
    const hasPropertyPath = triples.some(triple => triple.predicate.type === 'path')
    if (hasPropertyPath) occurences++
        
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 93


## wdt:P31/wdt:P279*

In [13]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const triples = parsedQuery.where.reduce(
        (triples, clause) => clause.type === 'bgp' ? triples.concat(clause.triples) : triples,
        []
    )
    const hasInstanceOfSubclassOf = triples.some(triple => {
        return ( triple.predicate.type === 'path'
            && triple.predicate.pathType === '/'
            && triple.predicate.items[0] === 'http://www.wikidata.org/prop/direct/P31'
            && triple.predicate.items[1].pathType === '*'
            && triple.predicate.items[1].items[0] === 'http://www.wikidata.org/prop/direct/P279' )
    })
    if (hasInstanceOfSubclassOf) occurences++
        
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 8


## wdt:P279*

In [14]:
var occurences = 0
var total = 0

var hasSubclassOf = function hasSubclassOf(predicate) {
    if (predicate.type !== 'path') return false
    if (predicate.pathType === '*' && predicate.items.includes('http://www.wikidata.org/prop/direct/P279')) {
        return true
    }
    
    return predicate.items.reduce((any, predicate) => { return any || hasSubclassOf(predicate) }, false)
}

eachFileInDir(EN_SPARQL, (query, resolve) => {
    const parsedQuery = parser.parse(query)
    const triples = parsedQuery.where.reduce(
        (triples, clause) => clause.type === 'bgp' ? triples.concat(clause.triples) : triples,
        []
    )
    const hasInstanceOfSubclassOf = triples.some(triple => hasSubclassOf(triple.predicate))
    if (hasInstanceOfSubclassOf) occurences++
        
    total++
    
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

992 53


## Use of Qualifiers

In [15]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    if (query.includes('pq:P')) occurences++

    total++
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

997 37


## Use of References

In [16]:
var occurences = 0
var total = 0

eachFileInDir(EN_SPARQL, (query, resolve) => {
    if (query.includes('pr:P')) occurences++

    total++
    resolve()
}).then(() => {
    console.log(total, occurences)
})

Promise { <pending> }

997 4


In [17]:
var predicates = {}

eachFileInDir(EN_SPARQL, (query, resolve) => {
    var wikibasePredicates = query.match(/wikibase:\w+/g)
    if (wikibasePredicates) wikibasePredicates.forEach((predicate) => {
        if (predicates[predicate]) predicates[predicate]++
        else predicates[predicate] = 1
    })
    resolve()
}).then(() => {
    var tuples = []
    for (var predicate in predicates) tuples.push([predicate, predicates[predicate]])
    console.log(tuples.sort((a, b) => b[1] - a[1]).slice(0, 5))
})

Promise { <pending> }

[ [ 'wikibase:label', 103 ],
  [ 'wikibase:language', 103 ],
  [ 'wikibase:statements', 15 ],
  [ 'wikibase:geoLatitude', 3 ],
  [ 'wikibase:geoLongitude', 3 ] ]
